Colab Link: https://colab.research.google.com/drive/1Ron53uVCclA0_86ej8iei8S-uGj19JIe#scrollTo=vU8oFSVBpuED

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf

2024-03-15 03:11:32.928913: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX512F
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-15 03:11:33.576841: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:460] Initializing the SageMaker Profiler.
2024-03-15 03:11:33.583823: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:105] SageMaker Profiler is not enabled. The timeline writer thread will not be started, future recorded events will be dropped.
2024-03-15 03:11:33.939965: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:460] Initializing the SageMaker Profiler.


In [ ]:
tf.__version__

'2.11.0'

## Artificial Neural Network

### Data Prep

#### Read

In [ ]:
dataset = pd.read_csv('ChurnData.csv')
dataset.head(2)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0


In [ ]:
X = dataset.iloc[:, 3:-1].values  # Keep only relevant columns
y = dataset.iloc[:, -1].values

In [ ]:
print(X)

[[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]


In [ ]:
print(y)

[1 0 1 ... 1 1 0]


#### Encoding categorical data

In [ ]:
# Label Encoding the "Gender" column
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:, 2] = le.fit_transform(X[:, 2])

In [ ]:
print(X)

[[619 'France' 0 ... 1 1 101348.88]
 [608 'Spain' 0 ... 0 1 112542.58]
 [502 'France' 0 ... 1 0 113931.57]
 ...
 [709 'France' 0 ... 0 1 42085.58]
 [772 'Germany' 1 ... 1 0 92888.52]
 [792 'France' 0 ... 1 0 38190.78]]


In [ ]:
# One Hot Encoding the "Geography" column
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
X = np.array(ct.fit_transform(X))

In [ ]:
print(X)

[[1.0 0.0 0.0 ... 1 1 101348.88]
 [0.0 0.0 1.0 ... 0 1 112542.58]
 [1.0 0.0 0.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 0.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 0.0 ... 1 0 38190.78]]


### Train Test Split

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

### Feature Scaling

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
# X_test = sc.transform(X_test)

### Build Model and Predict

In [ ]:
# Initializing the ANN
dl_ann = tf.keras.models.Sequential()

2024-03-15 03:12:03.519572: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX512F
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-15 03:12:03.526699: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [ ]:
# Adding the input layer and the first hidden layer
# units - neurons in hidden layer. its a hyperparameter and is determined by trial and error
# relu is the rectifier activation function
dl_ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

In [ ]:
# Adding the second hidden layer
dl_ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

In [ ]:
# Adding the output layer
# Since we are doing binary classification, the units in the output layer is set to 1
dl_ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

In [ ]:
# Compile the ANN with an optimizer and loss function
# Adam optimizer performs Stochastic Gradient Descent
dl_ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [ ]:
# Training the ANN on the Training set
# batch size and epochs are hyperparameters
dl_ann.fit(X_train, y_train, batch_size = 32, epochs = 100)

Epoch 1/100
250/250 [==============================] - 2s 3ms/step - loss: 0.5188 - accuracy: 0.7903
Epoch 2/100
250/250 [==============================] - 1s 3ms/step - loss: 0.4519 - accuracy: 0.7960
Epoch 3/100
250/250 [==============================] - 1s 3ms/step - loss: 0.4347 - accuracy: 0.7947
Epoch 4/100
250/250 [==============================] - 1s 3ms/step - loss: 0.4257 - accuracy: 0.7984
Epoch 5/100
250/250 [==============================] - 1s 3ms/step - loss: 0.4195 - accuracy: 0.8000
Epoch 6/100
250/250 [==============================] - 1s 3ms/step - loss: 0.4136 - accuracy: 0.8051
Epoch 7/100
250/250 [==============================] - 1s 2ms/step - loss: 0.4069 - accuracy: 0.8111
Epoch 8/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3973 - accuracy: 0.8229
Epoch 9/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3856 - accuracy: 0.8322
Epoch 10/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3750 - accura

In [ ]:
y_pred_ann = dl_ann.predict(sc.transform(X_test))
# Convert predicted probability to prediction
y_pred_ann = (y_pred_ann > 0.5)

63/63 [==============================] - 0s 3ms/step


### Check Predictions

In [ ]:
# Convert test and predicted arrays to vertical arrays
print("Test array shape before change: ", y_test.shape)
y_test_rshp = y_test.reshape(len(y_test),1)
print("Test array shape after change: ", y_test_rshp.shape)
print("\nPredicted array shape before change: ", y_pred_ann.shape)
y_pred_ann_rshp = y_pred_ann.reshape(len(y_pred_ann),1)
print("Predicted array shape after change: ", y_pred_ann_rshp.shape)

Test array shape before change:  (2000,)
Test array shape after change:  (2000, 1)

Predicted array shape before change:  (2000, 1)
Predicted array shape after change:  (2000, 1)


In [ ]:
# Print test and pred arrays side by side
comp_df = pd.DataFrame(np.concatenate((y_test_rshp,y_pred_ann),axis=1))
comp_df.columns = ['test','pred']
comp_df

,test,pred
0,0,0
1,1,0
2,0,0
3,0,0
4,0,0
...,...,...
1995,0,0
1996,0,0
1997,0,0
1998,0,0


### Confusion Matrix

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm_ann = confusion_matrix(y_test, y_pred_ann)
print("Confusion Matrix: ",cm_ann)
print("Number of correct predictions for label 0: ",cm_ann[0][0])
print("Number of correct predictions for label 1: ",cm_ann[1][1])
print("Number of incorrect predictions for label 0: ",cm_ann[0][1])
print("Number of incorrect predictions for label 1: ",cm_ann[1][0])

Confusion Matrix:  [[1502   93]
 [ 181  224]]
Number of correct predictions for label 0:  1502
Number of correct predictions for label 1:  224
Number of incorrect predictions for label 0:  93
Number of incorrect predictions for label 1:  181


In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred_ann)

0.863

## Convolutional Neural Network

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

### Data Prep

#### Read

In [ ]:
# Preprocess training set
# Image Augmentation - Transform data to avoid overfitting
train_datagen = ImageDataGenerator(rescale = 1./255,     # Feature scale all pixels (divides by 255)
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
training_set = train_datagen.flow_from_directory('data/CNNData1/training_set',
                                            target_size = (64, 64),   # Final size of images before feeding to CNN; smaller size results in faster training
                                            batch_size = 32,          # Number of images in each batch
                                            class_mode = 'binary')    # cat/dog in this example

Found 8000 images belonging to 2 classes.


In [ ]:
# Preprocess test set
# Only apply feature scaling to the test set. No transformations applied.
test_datagen = ImageDataGenerator(rescale = 1./255)
test_set = test_datagen.flow_from_directory('data/CNNData1/test_set',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'binary')

Found 1998 images belonging to 2 classes.


### Build Model

In [ ]:
# Initializing the CNN
cnn = tf.keras.models.Sequential()

In [ ]:
# Step1 - Add Convolution Layer
# input shae is needed only for the first layer
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[64, 64, 3]))

In [ ]:
# Step2 - Pooling
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

In [ ]:
# Add a second convolution and pooling layer
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

In [ ]:
# Step3 - Flattening
cnn.add(tf.keras.layers.Flatten())

In [ ]:
# Full Connection
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))

In [ ]:
# Output Layer
cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

In [ ]:
# Compile the CNN with an optimizer and loss function
cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

### Train and Evaluate

In [ ]:
# Train the CNN on the Training set and evaluate using the test set
# The first set of loss and accuracy values in the epoch output corresponds to training data and the second set corresponds to test data
cnn.fit(x = training_set, validation_data = test_set, epochs = 25)

Epoch 1/25
250/250 [==============================] - 63s 249ms/step - loss: 0.6758 - accuracy: 0.5745 - val_loss: 0.6768 - val_accuracy: 0.5986
Epoch 2/25
250/250 [==============================] - 60s 240ms/step - loss: 0.6155 - accuracy: 0.6611 - val_loss: 0.5921 - val_accuracy: 0.6757
Epoch 3/25
250/250 [==============================] - 65s 259ms/step - loss: 0.5720 - accuracy: 0.7024 - val_loss: 0.5334 - val_accuracy: 0.7402
Epoch 4/25
250/250 [==============================] - 59s 233ms/step - loss: 0.5381 - accuracy: 0.7296 - val_loss: 0.5442 - val_accuracy: 0.7162
Epoch 5/25
250/250 [==============================] - 54s 218ms/step - loss: 0.5060 - accuracy: 0.7526 - val_loss: 0.5043 - val_accuracy: 0.7568
Epoch 6/25
250/250 [==============================] - 54s 218ms/step - loss: 0.4824 - accuracy: 0.7670 - val_loss: 0.4930 - val_accuracy: 0.7653
Epoch 7/25
250/250 [==============================] - 54s 215ms/step - loss: 0.4807 - accuracy: 0.7620 - val_loss: 0.5211 - val_ac

### Predict

In [1]:
import keras.utils as image

In [ ]:
test_image = image.load_img('data/CNNData1/single_pred/cat_or_dog1.jpg', target_size = (64, 64)) # set size to 64 * 64 so that it matches the size used during training
test_image = image.img_to_array(test_image)    # convert to array
test_image = np.expand_dims(test_image, axis = 0)  # add a extra dummy dimension that will correspond to the batch size so that the format matches what the predict method expects 
result = cnn.predict(test_image)
training_set.class_indices  # allows us to know which index correspondes to which output. In this case, it will show that 1 is dog and 0 is cat
if result[0][0] == 1:
  prediction = 'dog'
else:
  prediction = 'cat'
print(prediction)

1/1 [==============================] - 0s 160ms/step
cat


In [ ]:
test_image = image.load_img('data/CNNData1/single_pred/cat_or_dog2.jpg', target_size = (64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = cnn.predict(test_image)
training_set.class_indices
if result[0][0] == 1:
  prediction = 'dog'
else:
  prediction = 'cat'
print(prediction)

1/1 [==============================] - 0s 29ms/step
dog
